# Projeto 1

João Antônio Misson Milhorim - 11834331

Reynaldo Coronatto Neto - 12547594

## Configurações Iniciais

In [7]:
import math
import glfw
import random
import numpy as np
import OpenGL.GL.shaders
import os

from PIL import Image
from OpenGL.GL import *

In [8]:
class GLWrapper:
    ''' Classe que faz o setup do OpenGL e permite a visualização de imagens '''
        
    def __init__(self, height=700, width=700, title='', figures=None):
        ''' Construtor responsável por chamar funções que configuram o OpenGL '''
        self.show_object = False
        self.figures = figures
        self.set_keys()
        self.set_increments()
        self.window = self.init_window(height, width, title)
        self.add_events(self.window)
        self.program, self.vertex, self.fragment = self.set_shaders()
        self.compile_shaders(self.vertex, 'vertex')
        self.compile_shaders(self.fragment, 'fragment')
        self.attach_shaders(self.program, self.vertex, self.fragment)
        self.link_program(self.program)
    
    def set_keys(self):
        self.KEY_ARROW_DOWN = 264
        self.KEY_ARROW_UP = 265
        self.KEY_ARROW_LEFT = 263
        self.KEY_ARROW_RIGHT = 262
        self.KEY_A = 65
        self.KEY_S = 83
        self.KEY_D = 68
        self.KEY_W = 87
        self.KEY_X = 88
        self.KEY_Z = 90
        self.KEY_V = 86
        self.KEY_P = 80
        
        self.KEY_1 = 49
        self.KEY_2 = 50
        self.KEY_3 = 51
        self.KEY_4 = 52
        self.KEY_5 = 53
    
    def set_increments(self):
        self.rx_inc = 0.0
        self.ry_inc = 0.0
        self.s_inc = 0.0
        self.tx_inc = 0.0
        self.ty_inc = 0.0
        self.polygonal = False
        self.magnification = False
    
    def init_window(self, height, width, title):
        ''' Inicializa janela '''
        glfw.init()
        glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
        window = glfw.create_window(height, width, title, None, None)
        glfw.make_context_current(window)
        
        return window
        
    def key_event(self, window, key, scancode, action, mods, figure_id):
        ''' Callback para cliques no teclado '''
        if action == 0:
            return
        
        if key == self.KEY_W: # sobe objeto no eixo y
            self.ty_inc += 0.0005
        
        if key == self.KEY_S: # desce objeto no eixo y
            self.ty_inc -= 0.0005
        
        if key == self.KEY_A: # objeto para a esquerda
            self.tx_inc -= 0.0005
        
        if key == self.KEY_D: # objeto para direita
            self.tx_inc += 0.0005
        
        if key == self.KEY_ARROW_RIGHT: # rotaciona objeto em torno do eixo y
            self.ry_inc -= 0.001
        
        if key == self.KEY_ARROW_LEFT: # rotaciona objeto em torno do eixo y
            self.ry_inc += 0.001
        
        if key == self.KEY_ARROW_UP: # rotaciona objeto em torno do eixo x
            self.rx_inc += 0.001
        
        if key == self.KEY_ARROW_DOWN: # rotaciona objeto em torno do eixo x
            self.rx_inc -= 0.001
        
        if key == self.KEY_Z: # diminui escala do objeto
            self.s_inc -= 0.001
        
        if key == self.KEY_X: # aumenta escala do objeto
            self.s_inc += 0.001
        
        if key == self.KEY_P and action == 1: # ativar ou desativar modo poligonal
            self.polygonal = not self.polygonal
        
        if key == self.KEY_V and action == 1: # ativar ou desativar modo poligonal
            self.maginification = not self.magnification
            
    
    def add_events(self, window):
        ''' Adiciona callbacks dos eventos '''
        glfw.set_key_callback(window, self.key_event)
    
    def set_shaders(self):
        ''' Adiciona shaders '''
        vertex_code = """
                attribute vec3 position;
                attribute vec2 texture_coord;
                varying vec2 out_texture;

                uniform mat4 mat_transform;        

                void main(){
                    gl_Position = mat_transform * vec4(position,1.0);
                    out_texture = vec2(texture_coord);
                }
                """

        fragment_code = """
                uniform vec4 color;
                varying vec2 out_texture;
                uniform sampler2D samplerTexture;

                void main(){
                    vec4 texture = texture2D(samplerTexture, out_texture);
                    gl_FragColor = texture;
                }
        """

        program  = glCreateProgram()
        vertex   = glCreateShader(GL_VERTEX_SHADER)
        fragment = glCreateShader(GL_FRAGMENT_SHADER)
        
        glShaderSource(vertex, vertex_code)
        glShaderSource(fragment, fragment_code)
        
        return program, vertex, fragment

    def compile_shaders(self, shader, name):
        ''' Compila shaders '''
        glCompileShader(shader)
        if not glGetShaderiv(shader, GL_COMPILE_STATUS):
            error = glGetShaderInfoLog(shader).decode()
            print(error)
            raise RuntimeError(f"Erro de compilacao do {name} Shader")
            
    def attach_shaders(self, program, vertex, fragment):
        ''' Associa shaders '''
        glAttachShader(program, vertex)
        glAttachShader(program, fragment)
        
    def link_program(self, program):
        ''' Faz o link do programa '''
        glLinkProgram(program)
        if not glGetProgramiv(program, GL_LINK_STATUS):
            print(glGetProgramInfoLog(program))
            raise RuntimeError('Linking error')

        glUseProgram(program)
    
    def insert_figure_gpu(self, vertices, textures, mat_transform=None):
        ''' Gera um buffer e insere dados vertices e texturas na janela, utiliza matriz
            de transformação, caso seja informada '''
        buffer = glGenBuffers(2)
        
        # vertices
        glBindBuffer(GL_ARRAY_BUFFER, buffer[0])

        glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
        glBindBuffer(GL_ARRAY_BUFFER, buffer)

        stride = vertices.strides[0]
        offset = ctypes.c_void_p(0)

        loc = glGetAttribLocation(self.program, "position")
        glEnableVertexAttribArray(loc)

        glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)
        
        
        # texturas
        glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
        glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
        
        stride = textures.strides[0]
        offset = ctypes.c_void_p(0)
        
        loc_texture_coord = glGetAttribLocation(program, "texture_coord")
        glEnableVertexAttribArray(loc_texture_coord)
        
        glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

    def apply_transformations(self):
        self.figures[i].update_mat_tranform(self.rx_inc, self.ry_inc, 
                                         self.s_inc, self.tx_inc, self.ty_inc)
        mat = self.figures.get_mat_transform()
        
        loc = glGetUniformLocation(self.program, "mat_transformation")
        glUniformMatrix4fv(loc, 1, GL_TRUE, mat)

In [ ]:
def matrix_multiplication(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

In [9]:
class Object:
    ''' Classe para lidar com objetdos 3D'''
    
    def __init__(self):
        self.dx = 0.0 
        self.dy = 0.0
        
        self.tx = 0.0
        self.ty = 0.0
        
        self.scale = 0.01
        
        self.mat_transform = np.array([ 1.0, 0.0, 0.0, 0.0, 
                                        0.0, 1.0, 0.0, 0.0, 
                                        0.0, 0.0, 1.0, 0.0, 
                                        0.0, 0.0, 0.0, 1.0], np.float32)
    
    def update_mat_transform(self, rx_inc, ry_inc, s_inc, tx_inc, ty_inc):
        self.dx += rx_inc
        cos_dx = math.cos(self.dx)
        sin_dx = math.sin(self.dx)
        
        rot_x = np.array([  1.0, 0.0, 0.0, 0.0, 
                            0.0, cos_dx, -sin_dx, 0.0, 
                            0.0, sin_dx, cos_dx, 0.0, 
                            0.0, 0.0, 0.0, 1.0], np.float32)
        
        self.dy += ry_inc
        cos_dy = math.cos(self.dy)
        sin_dy = math.sin(self.dy)
        
        rot_y np.array([ cos_dy, 0.0, sin_dy, 0.0, 
                         0.0, 1.0, 0.0, 0.0, 
                         -sin_dy, 0.0, cos_dy, 0.0, 
                         0.0, 0.0, 0.0, 1.0], np.float32)
        
        self.scale += s_inc
        scale = np.array([  self.scale, 0.0, 0.0, 0.0, 
                            0.0, self.scale, 0.0, 0.0, 
                            0.0, 0.0, self.scale, 0.0, 
                            0.0, 0.0, 0.0, 1.0], np.float32)
        
        self.tx += tx_inc
        self.ty += ty_inc
        
        translation = np.array([1.0, 0.0, 0.0, self.t_x, 
                                0.0, 1.0, 0.0, self.t_y, 
                                0.0, 0.0, 1.0, 0.0, 
                                0.0, 0.0, 0.0, 1.0], np.float32)
        
        self.mat_transform = matrix_multiplication(mat_rotation_y, mat_rotation_x)
        self.mat_transform = matrix_multiplication(mat_scale, self.mat_transform)
        self.mat_transform = matrix_multiplication(mat_translation, self.mat_transform)
    
    
    def get_mat_transform(self):
        return self.mat_transform
    
    def set_textures(self, qnt_texturas=10):
        glEnable(GL_TEXTURE_2D)
        textures = glGenTextures(qtd_texturas)
    
    def load_model_from_file(filename):
        ''' Loads a Wavefront OBJ file. '''
        objects = {}
        vertices = []
        texture_coords = []
        faces = []
        max_values = []

        material = None

        # abre o arquivo obj para leitura
        for line in open(filename, "r"): ## para cada linha do arquivo .obj
            if line.startswith('#'): continue ## ignora comentarios
            values = line.split() # quebra a linha por espaço
            if not values: continue


            ### recuperando vertices
            if values[0] == 'v':
                vertices.append(values[1:4])


            ### recuperando coordenadas de textura
            elif values[0] == 'vt':
                texture_coords.append(values[1:3])

            ### recuperando faces 
            elif values[0] in ('usemtl', 'usemat'):
                material = values[1]
            elif values[0] == 'f':
                face = []
                face_texture = []
                for v in values[1:]:
                    w = v.split('/')
                    face.append(int(w[0]))
                    if len(w) >= 2 and len(w[1]) > 0:
                        face_texture.append(int(w[1]))
                    else:
                        face_texture.append(0)

                faces.append((face, face_texture, material))

        model = {}
        model['vertices'] = vertices
        model['texture'] = texture_coords
        model['faces'] = faces
        
        return model
    
    def load_texture_from_file(texture_id, img_textura):
        glBindTexture(GL_TEXTURE_2D, texture_id)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
        img = Image.open(img_textura)
        img_width = img.size[0]
        img_height = img.size[1]
        image_data = img.tobytes("raw", "RGB", 0, -1)
        #image_data = np.array(list(img.getdata()), np.uint8)
        glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)
        
    def load_object():
        vertices_list = []    
        textures_coord_list = []
        
        modelo = load_model_from_file('Cat.obj')

        ### inserindo vertices do modelo no vetor de vertices
        print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
        for face in modelo['faces']:
            for vertice_id in face[0]:
                vertices_list.append( modelo['vertices'][vertice_id-1] )
            for texture_id in face[1]:
                textures_coord_list.append( modelo['texture'][texture_id-1] )
        print('Processando modelo cube.obj. Vertice final:',len(vertices_list))

        ### inserindo coordenadas de textura do modelo no vetor de texturas


        ### carregando textura equivalente e definindo um id (buffer): use um id por textura!
        load_texture_from_file(0,'Cat.jpg')

In [10]:
g = GLWrapper(title='Projeto 1')

In [11]:
glfw.show_window(g.window)

In [12]:
glEnable(GL_DEPTH_TEST)

while not glfw.window_should_close(g.window):
    glfw.poll_events()

    g.apply_transformation()
    
    
    
    glfw.swap_buffers(g.window)

glfw.terminate()